In [19]:
from cbpw_liba import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загружаем данные для каждого домена
NUM_OF_DOMENS = 14

progress = widgets.IntProgress(min=0, max=NUM_OF_DOMENS, description='Progress:', bar_style='info')
status = widgets.Label(value="Инициализация...")
box = widgets.VBox([progress, status])
display(box)

population, clear_population = load_hdf5('processed_data/population'), load_hdf5('processed_data/clear_population')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Население"
labor, clear_labor = load_hdf5('processed_data/labor'), load_hdf5('processed_data/clear_labor')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Труд"
wellbeing, clear_wellbeing = load_hdf5('processed_data/wellbeing'), load_hdf5('processed_data/clear_wellbeing')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Уровень жизни населения"
education, clear_education = load_hdf5('processed_data/education'), load_hdf5('processed_data/clear_education')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Образование"
healthcare, clear_healthcare = load_hdf5('processed_data/healthcare'), load_hdf5('processed_data/clear_healthcare')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Здравоохранение"
culture, clear_culture = load_hdf5('processed_data/culture'), load_hdf5('processed_data/clear_culture')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Культура отдых и туризм"
nature_and_reserve, clear_nature_and_reserve = load_hdf5('processed_data/nature_and_reserve'), load_hdf5('processed_data/clear_nature_and_reserve')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Земля и охрана природы"
founds, clear_founds = load_hdf5('processed_data/founds'), load_hdf5('processed_data/clear_founds')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Основные фонды"
investments, clear_investments = load_hdf5('processed_data/investments'), load_hdf5('processed_data/clear_investments')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Инвестиции"
goods_and_services, clear_goods_and_services = load_hdf5('processed_data/goods_and_services'), load_hdf5('processed_data/clear_goods_and_services')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Торговля и услуги населению"
transport, clear_transport = load_hdf5('processed_data/transport'), load_hdf5('processed_data/clear_transport')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Транспорт"
r_and_d, clear_r_and_d = load_hdf5('processed_data/r_and_d'), load_hdf5('processed_data/clear_r_and_d')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Наука и инновации"
finance, clear_finance = load_hdf5('processed_data/finance'), load_hdf5('processed_data/clear_finance')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Финансы"
prices_and_tariffes, clear_prices_and_tariffes = load_hdf5('processed_data/prices_and_tariffes'), load_hdf5('processed_data/clear_prices_and_tariffes')
progress.value = progress.value + 1
status.value = "✅ Обработан домен: Цены и тарифы"

# Создаем словарь с результатами
dict_of_final_results = {
    "Население": [population, clear_population],
    "Труд": [labor, clear_labor],
    "Уровень жизни населения": [wellbeing, clear_wellbeing],
    "Образование": [education, clear_education],
    "Здравоохранение": [healthcare, clear_healthcare],
    "Культура отдых и туризм": [culture, clear_culture],
    "Земля и охрана природы": [nature_and_reserve, clear_nature_and_reserve],
    "Основные фонды": [founds, clear_founds],
    "Инвестиции": [investments, clear_investments],
    # "Предприятия и организации": [enterprises, clear_enterprises],
    # "Сельхоз и рыболовство": [agriculture, clear_agriculture],
    # "Строительство": [building, clear_building],
    "Торговля и услуги населению": [goods_and_services, clear_goods_and_services],
    "Транспорт": [transport, clear_transport],
    "Наука и инновации": [r_and_d, clear_r_and_d],
    "Финансы": [finance, clear_finance],
    "Цены и тарифы": [prices_and_tariffes, clear_prices_and_tariffes]
}

############################ [START] EXPERIMENTAL ZONE ####################################
status.value = "⏳ Надстройка базы..."
for domen_name, (domen, clear_domen) in dict_of_final_results.items():
    method = 1
    if domen_name == "Финансы":
        method = 2
    index_share = 0.9 # учет долгосрочного индекса
    base_year = 2012 # базовый год
    domen = {year : domen[year][domen[year]['object_level'] == 'регион'] for year in domen.keys()}
    for year, df in domen.items():
        domen[year] = drop_missing_values(df, domen[list(domen.keys())[0]].columns)
    domen_norm_by_current = normalized_dict_by_minmax(domen)
    domen_norm_by_base = normalized_dict_by_minmax_by_base_year(domen, base_year)
    domen_indexes_by_base, weights_by_base = calculate_index_with_weights(domen_norm_by_base, method)
    domen_indexes_by_current, weights_by_current = calculate_index_with_weights(domen_norm_by_current, method)
    domen_final = combine_indices(domen_indexes_by_base, domen_indexes_by_current, index_share)
    if 'Москва' in domen_final.index:
        domen_final = domen_final.drop(index=['Москва']) # Выкидываем Москву
    # print(domen_name, f"| Пропущено: {domen_final.isna().sum().sum()}")
    domen_final = domen_final.fillna(domen_final.mean())
    dict_of_final_results[domen_name].append(domen_final)

status.value = "🎉 Все домены успешно обработаны!"
progress.bar_style = 'success'

############################ [END] EXPERIMENTAL ZONE ####################################



# for domen_name, (domen, clear_domen) in dict_of_final_results.items():
#     method = 1
#     if domen_name == "Финансы":
#         method = 2
#     index_share = 0.9 # учет долгосрочного индекса
#     base_year = 2012 # базовый год
#     domen = {year : domen[year][domen[year]['object_level'] == 'регион'] for year in domen.keys()}
#     for year, df in domen.items():
#         domen[year] = drop_missing_values(df, domen[list(domen.keys())[0]].columns)
#     domen_norm_by_current = normalized_dict_by_minmax(domen)
#     domen_norm_by_base = normalized_dict_by_minmax_by_base_year(domen, base_year)
#     domen_indexes_by_base, weights_by_base = calculate_index_with_weights(domen_norm_by_base, method)
#     domen_indexes_by_current, weights_by_current = calculate_index_with_weights(domen_norm_by_current, method)
#     domen_final = combine_indices(domen_indexes_by_base, domen_indexes_by_current, index_share)
#     if 'Москва' in domen_final.index:
#         domen_final = domen_final.drop(index=['Москва']) # Выкидываем Москву
#     # print(domen_name, f"| Пропущено: {domen_final.isna().sum().sum()}")
#     domen_final = domen_final.fillna(domen_final.mean())
#     dict_of_final_results[domen_name].append(domen_final)
# # assert(0 == sum(value[-1].isna().sum().sum() for key, value in dict_of_final_results.items())) # "Не все пропуски были заполнены!!!"

social = [
    "Население",
    "Труд",
    "Уровень жизни населения",
    "Образование",
    "Здравоохранение",
    "Культура отдых и туризм",
    "Земля и охрана природы",
    "Основные фонды",
    "Инвестиции",
    # "Предприятия и организации",
    # "Сельхоз и рыболовство",
    # "Строительство",
    "Торговля и услуги населению",
    "Транспорт",
    "Наука и инновации",
    "Цены и тарифы"
]

# Financial Integral Index of Region Development
FIIRD = norm_final_domen_by_minmax_normalizing(dict_of_final_results['Финансы'][-1])

# Social Integral Index of Region Development
decomposed_SIIRD = {domen : norm_final_domen_by_minmax_normalizing(dict_of_final_results[domen][-1]) for domen in dict_of_final_results.keys()}
decomposed_SIIRD.pop("Финансы")
SIIRD = compose_SIIRD(decomposed_SIIRD)

# Social and Financial Integral Index of Region Development
SFIIRD = {"SIIRD" : SIIRD, "FIIRD" : dict_of_final_results['Финансы'][-1]}
SFIIRD = to_one_structure(SFIIRD)
SFIIRD = {key : norm_final_domen_by_minmax_normalizing(value) for key, value in SFIIRD.items()}
SFIIRD = compose_SFIIRD(SFIIRD, alpha=0.5)

out = widgets.Output(layout={'border': '1px solid lightgray'})

def update_plot(change=None):
    region = region_dropdown.value
    alpha  = alpha_slider.value
    
    final_index = alpha * FIIRD + (1 - alpha) * SIIRD
    
    FIIRD_df = FIIRD.loc[region]
    SIIRD_df = SIIRD.loc[region]
    years       = FIIRD_df.index.astype(int)
    fiird_vals  = FIIRD_df.values
    siird_vals  = SIIRD_df.values
    final_vals  = final_index.loc[region].values
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=years, y=fiird_vals,  mode='lines+markers', name='FIIRD'))
    fig.add_trace(go.Scatter(x=years, y=siird_vals,  mode='lines+markers', name='SIIRD'))
    fig.add_trace(go.Scatter(x=years, y=final_vals,  mode='lines+markers', name='SFIIRD',
                             line=dict(dash='dash', width=3)))
    fig.update_layout(
        title=f"Динамика индексов: {region}",
        xaxis_title="Год", yaxis_title="Индекс",
        template="plotly_white", height=500
    )
    
    with out:
        clear_output(wait=True)
        display(fig)

region_dropdown = widgets.Dropdown(
    options=sorted(SIIRD.index.tolist()),
    value=SIIRD.index.tolist()[0],
    description="Регион:"
)
alpha_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.01,
    description="Alpha:"
)

region_dropdown.observe(update_plot, names='value')
alpha_slider.observe(update_plot, names='value')

update_plot()

ui = widgets.VBox([widgets.HBox([region_dropdown, alpha_slider]), out])
# display(ui)

dataframes = {'SIIRD': SIIRD, 'FIIRD': FIIRD, 'SFIIRD': SFIIRD}

out2 = widgets.Output(layout={'border': '1px solid lightgray'})

def update_index_plot(change=None):
    idx      = index_dropdown.value
    regions  = regions_select.value or []
    stat     = statistic_choice.value
    
    df       = dataframes[idx]
    years    = [int(c) for c in df.columns if str(c).isdigit()]
    fig      = go.Figure()
    
    if stat == 'mean':
        fig.add_trace(go.Bar(x=years, y=df[years].mean(), name="Среднее по регионам", opacity=0.6))
    else:
        fig.add_trace(go.Bar(x=years, y=df[years].median(), name="Медиана по регионам", opacity=0.6))
    
    colors = ['green', 'red', 'orange', 'purple', 'brown', 'blue', 'pink']
    for i, region in enumerate(regions):
        if region in df.index:
            fig.add_trace(
                go.Scatter(x=years,
                           y=df.loc[region, years],
                           mode='lines+markers',
                           name=region,
                           line=dict(color=colors[i % len(colors)], width=2),
                           marker=dict(size=6))
            )
    
    fig.update_layout(
        title=f"Динамика индекса {idx}",
        xaxis_title="Год", yaxis_title="Индекс",
        template='plotly_white',
        height=600,
        hovermode="closest"
    )
    
    with out2:
        clear_output(wait=True)
        display(fig)

index_dropdown = widgets.Dropdown(
    options=list(dataframes.keys()),
    value='SIIRD',
    description='Индекс:',
    style={'description_width': 'initial'}
)

regions_select = widgets.SelectMultiple(
    options=sorted(dataframes['SIIRD'].index.tolist()),
    value=['Московская область', 'Нижегородская область'],
    description='Регионы:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

statistic_choice = widgets.RadioButtons(
    options=[('Среднее', 'mean'), ('Медиана', 'median')],
    value='mean',
    description='Статистика:',
    style={'description_width': 'initial'}
)

def on_index_change(change):
    new_idx = change['new']
    regions_select.options = sorted(dataframes[new_idx].index.tolist())
    update_index_plot()

index_dropdown.observe(on_index_change, names='value')
regions_select.observe(update_index_plot, names='value')
statistic_choice.observe(update_index_plot, names='value')

update_index_plot()

ui2 = widgets.VBox([
    widgets.HBox([index_dropdown, statistic_choice]),
    regions_select,
    out2
])
# display(ui2)

domen_mapping = {
    "Население": ("population", "clear_population"),
    "Труд": ("labor", "clear_labor"),
    "Уровень жизни населения" : ("wellbeing", "clear_wellbeing"),
    "Образование": ("education", "clear_education"),
    "Здравоохранение" : ("healthcare", "clear_healthcare"),
    "Культура отдых и туризм": ("culture", "clear_culture"),
    "Земля и охрана природы" : ("nature_and_reserve", "clear_nature_and_reserve"),
    "Основные фонды" : ("founds", "clear_founds"),
    "Инвестиции" : ("investments", "clear_investments"),
    # "Предприятия и организации" : ("enterprises", "clear_enterprises"),
    # "Сельхоз и рыболовство" : ("agriculture", "clear_agriculture"),
    # "Строительство" : ("building", "clear_building"),
    "Торговля и услуги населению" : ("goods_and_services", "clear_goods_and_services"),
    "Транспорт" : ("transport", "clear_transport"),
    "Наука и инновации" : ("r_and_d", "clear_r_and_d"),
    "Цены и тарифы" : ("prices_and_tariffes", "clear_prices_and_tariffes"),
    "Финансы" : ("finance", "clear_finance")
}

# Виджет для выбора домена
domen_widget = widgets.Dropdown(
    options=domen_mapping.keys(),
    description='Домен:'
)

# Виджеты для выбора параметров
region_widget = widgets.Dropdown(
    options=[],  # Будет заполняться при выборе домена
    description='Регион:'
)

method_calculation_widget = widgets.Dropdown(
    options = ['PCA', 'Обратная дисперсия', 'Равные веса'],
    value='PCA',  # Значение по умолчанию - PCA
    description='Метод расчета:'
)

base_year_widget = widgets.IntSlider(
    value=2012,
    min=2000,
    max=2022,
    step=1,
    description='Базовый год:'
)

index_share_widget = widgets.FloatSlider(
    value=0.9,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Alpha:'
)

# Кнопки управления
load_domen_button = widgets.Button(description="Загрузить Домен", button_style='primary')
interactive_analysis_button = widgets.Button(description="Интерактивный Анализ", button_style='info')
update_button = widgets.Button(description="Обновить данные", button_style='success')
show_table_button = widgets.Button(description="Показать таблицу", button_style='success')

# Поля вывода
output_domen = widgets.Output()
output_interactive = widgets.Output()
output_main = widgets.Output()
output_table = widgets.Output()

# Глобальные переменные
domen = None
clear_domen = None
domen_name = ""
region_name = ""
method_calculation = 1

# Функция выбора домена
def load_domen(_=None):
    global domen, clear_domen, domen_name, region_name
    domen_key = domen_widget.value
    domen_var, clear_domen_var = domen_mapping[domen_key]  # Получаем переменные

    # Устанавливаем глобальные переменные
    domen = globals()[domen_var]
    clear_domen = globals()[clear_domen_var]
    domen_name = domen_key  # Человекочитаемое имя домена

    # Обновляем список регионов
    region_widget.options = sorted(domen[list(domen.keys())[0]]['object_name'].unique())

    output_domen.clear_output()
    with output_domen:
        print(f"Выбран домен: {domen_name}")

load_domen_button.on_click(load_domen)

# Функция для интерактивного анализа
def interactive_analysis(_=None):
    global clear_domen
    output_interactive.clear_output()
    with output_interactive:
        clear_domen = {year : clear_domen[year][clear_domen[year]['object_level'] == 'регион'].sort_values(by='object_name') for year in clear_domen.keys()}
        analyze_indicator_interactive(clear_domen)

interactive_analysis_button.on_click(interactive_analysis)

# Основная обработка данных
def process_data(_=None):
    global domen, clear_domen, domen_name, region_name, method_calculation

    if domen is None:
        return
    region_name = region_widget.value
    base_year = base_year_widget.value
    index_share = index_share_widget.value
    # if domen_name == "Финансы":
    #     method_calculation_widget.options = ['Обратная дисперсия', 'Равные веса']
    calculation_method = {'PCA' : 1, 'Обратная дисперсия' : 2, 'Равные веса' : 3}
    method_calculation = calculation_method[method_calculation_widget.value]

    output_main.clear_output()

    with output_main:
        print(f"Обновление данных для региона: {region_name}, базовый год: {base_year}, индекс: {index_share}")

        # Фильтрация данных по региону
        domen_filtered = {year: domen[year][domen[year]['object_level'] == 'регион'] for year in domen.keys()}

        # Удаление пропусков
        for year, df in domen_filtered.items():
            domen_filtered[year] = drop_missing_values(df, domen_filtered[list(domen_filtered.keys())[0]].columns)

        # Нормализация по текущему году и базовому году
        domen_norm_by_current = normalized_dict_by_minmax(domen_filtered)
        domen_norm_by_base = normalized_dict_by_minmax_by_base_year(domen_filtered, base_year)

        # Расчет индексов и весов
        domen_indexes_by_base, weights_by_base = calculate_index_with_weights(domen_norm_by_base, method_calculation)
        domen_indexes_by_current, weights_by_current = calculate_index_with_weights(domen_norm_by_current, method_calculation)

        # Финальный индекс
        domen_final = combine_indices(domen_indexes_by_base, domen_indexes_by_current, index_share)

        # Построение графиков
        plot_index_trends_multi(domen_indexes_by_base, domen_indexes_by_current, domen_final, region_name, domen_name)
        plot_index_trends_with_similar(domen_indexes_by_base, domen_indexes_by_current, domen_final, region_name, domen_name, 3)
        plot_top_successful_regions_dynamic(domen_final, domen_name, 10, 0.1)
        plot_bottom_successful_regions_dynamic(domen_final, domen_name, 10, 0.1)

        # Оценка влияния компонентов
        domen_norm_base_weighted = {year: count_impact_per_component(domen_norm_by_base[year], weights_by_base) for year in domen_norm_by_base.keys()}
        domen_norm_current_weighted = {year: count_impact_per_component(domen_norm_by_current[year], weights_by_current) for year in domen_norm_by_current.keys()}
        global importances_of_domen
        importances_of_domen = combine_for_weights_and_importances(domen_norm_base_weighted, domen_norm_current_weighted, index_share)

update_button.on_click(process_data)

year_widget = widgets.IntSlider(
    value=2000,
    min=2000,
    max=2022,
    step=1,
    description='Год:'
)

def show_table(_=None):
    # Очищаем вывод графиков
    output_main.clear_output()

    output_table.clear_output()
    with output_table:
        selected_year = year_widget.value  # Получаем выбранный год
        print(f"Индекс региона: {region_widget.value}, Год: {selected_year}")  # Выводим индекс региона и выбранный год
        df_importances = rank_importances(clear_domen, domen, importances_of_domen, region_widget.value, selected_year)
        display(df_importances)

show_table_button.on_click(show_table)

clear_screen_button = widgets.Button(description="Очистить экран", button_style='danger')

# Функция очистки экрана
def clear_screen(_=None):
    # Очищаем все области вывода
    output_domen.clear_output()
    output_interactive.clear_output()
    output_main.clear_output()
    output_table.clear_output()

clear_screen_button.on_click(clear_screen)

# Отображение интерфейса с кнопкой очистки экрана
ui_main = VBox([
    HBox([domen_widget, load_domen_button]),
    output_domen,
    interactive_analysis_button,
    output_interactive,
    HBox([region_widget, base_year_widget, index_share_widget]),
    HBox([update_button, method_calculation_widget]),
    show_table_button,
    year_widget,  # Новый виджет для выбора года
    output_main,
    output_table,
    clear_screen_button  # Добавляем кнопку очистки экрана
])


tab = widgets.Tab(children=[ui, ui2, ui_main])
tab.set_title(0, '📈 SFIIRD Dynamics')
tab.set_title(1, '📊 Index Comparison')
tab.set_title(2, '🌐 Domain Analysis')
display(tab)